# Testing Pretrained ResNet101

1. Mount Google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

2. Change Runtime to "GPU"

3. Test a sample using the pre-trained ResNet101

In [2]:
proj_dir = "/content/gdrive/MyDrive/SUTD/Term7/DL/HW/HW3"

In [3]:
# https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/

import torch, torchvision
from PIL import Image

# load model
resnet = torchvision.models.resnet101(pretrained=True)

# set network to evaluation mode
resnet.eval()

transform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(256),
  # torchvision.transforms.CenterCrop(224),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
  )])


img = Image.open(f"{proj_dir}/animal.jpg")
img_t = transform(img)
print(img_t.shape)

In [7]:
# To check the cropping of images
import matplotlib.pyplot as plt
plt.imshow(img_t[:, (img_t.shape[1]-224)//2:(img_t.shape[1]+224)//2, (img_t.shape[2]-224)//2:(img_t.shape[2]+224)//2].permute(1, 2, 0))

In [5]:
crops = [
  img_t[:, (img_t.shape[1]-224)//2:(img_t.shape[1]+224)//2, (img_t.shape[2]-224)//2:(img_t.shape[2]+224)//2], # center
  img_t[:, :224, :224], # upper left
  img_t[:, (img_t.shape[1]-224):, :224], # lower left
  img_t[:, (img_t.shape[1]-224):, (img_t.shape[2]-224):], # lower right
  img_t[:, :224, (img_t.shape[2]-224):], # upper right
]

In [6]:
for idx, crop_t in enumerate(crops):
  batch_t = torch.unsqueeze(crop_t, 0)

  # perform inference
  out = resnet(batch_t)

  # print top-5 classes predicted by model
  _, indices = torch.sort(out, descending=True)
  if idx == 0:
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100 / 5
  else:
    percentage += torch.nn.functional.softmax(out, dim=1)[0] * 100 / 5
for idx in indices[0][:5]:
  print('Label:', idx, '. Confidence Score:', percentage[idx].item(), '%')

4. Refer to https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b, to check if the predicted classes are meaningful.

5. Task

Modify the code above, to peform data augmentation for the testing sample (averaging the scores of 5 crops: center crop, upper left crop, lower left crop, lower right crop, upper right crop).

Pls briefly discuss the advantages and disadvantages of using testing data augmentation.

Advantages of using testing data augmentation:

* Makes test data more similar to data which the model is trained on, possibly making predictions better.
* Testing data augmentation allows us to fix dimension issues when the model takes in fixed dimensions of input.
* Testing data augmentation can help increase the quality of testing inputs for better predictions.

Disadvantages of using testing data augmentation:

* Augmented data might not be representative of real-world input (e.g. mirroring an image with text)